In [ ]:
# Historical pp curve error calculation: Euclidean distance MSE

In [ ]:
import sys
sys.path.append('../..')
from pymongo import UpdateOne
from pymongo import MongoClient
from tqdm import tqdm
import numpy as np
from exploration.config import mongo_inst
from mlpp.data_collection.sample import osuDumpSampler
import datetime
from datetime import datetime
import pprint
import matplotlib.pyplot as plt
# from fastdtw import fastdtw
# from scipy.spatial.distance import euclidean

In [ ]:
client = MongoClient(port=27017)
db = client.osu_top_db
top_db = mongo_inst["osu_top_db"]
test = osuDumpSampler(top_db)
user_ids = list(map(lambda c: c['_id'], top_db['osu_user_stats'].find({}, {})))

In [ ]:
#Find users that have more than 50 data points & creates a new list of user ids
big_user_ids = []
subset_user_ids = user_ids[0:100] # the first ten had 5, the second had 6, the third had 5

for i in subset_user_ids:
    datapts = len(list(db.osu_track_updates.find({"user_id": i}, {"date":1})))
    if datapts > 50: #arbitrary number
        big_user_ids.append(i)


In [ ]:
#Finds the date & real_pp and then sorts by date
real_pp = {}
for i in big_user_ids:
    real_pp[i] = list(db.osu_track_updates.find( {"user_id": i}, {"_id":0, "timestamp": 1, "pp_raw":1}))
    real_pp[i].sort(key = lambda x:x["timestamp"])
#real_pp[big_user_ids[0]]

In [ ]:
#Finds the date and est_user_pp and then sorts by date
est_pp = {}
for i in big_user_ids:
    est_pp[i] = list(db.osu_scores_high.find( {"user_id": i}, {"_id":0, "date": 1, "mlpp.est_user_pp":1}))
    est_pp[i].sort(key = lambda x:x["date"]) 
#est_pp[big_user_ids[0]]

In [ ]:
#Finds the overlap time periods
time_comparison = {}
for i in big_user_ids:
    earliest_est_date = est_pp[i][0]["date"]
    earliest_real_date = real_pp[i][0]["timestamp"]
    latest_est_date = est_pp[i][-1]["date"]
    latest_real_date = real_pp[i][-1]["timestamp"]
    if earliest_est_date >= earliest_real_date:
        start = earliest_est_date
    else:
        start = earliest_real_date
    if latest_est_date <= latest_real_date:
        end = latest_est_date
    else:
        end = latest_real_date
    time_comparison[i] = {"start": start, "end": end}
#pp=pprint.PrettyPrinter(indent=4)
#pp.pprint(time_comparison)

In [ ]:
#Gets the table for real user pp
first_user = big_user_ids[0]
real_start_time = []
real_pp_points = []
for i in real_pp[first_user]:
    real_start_time.append(i["timestamp"])
    real_pp_points.append(i["pp_raw"])
real_table = np.column_stack((real_start_time, real_pp_points))
#plt.plot(real_table[:,0], real_table[:,1])


In [ ]:
#Gets the table for est user pp
est_start_time = []
est_pp_points = []
for i in est_pp[first_user]:
    est_start_time.append(i["date"])
    est_pp_points.append(i["mlpp"]["est_user_pp"])
est_table = np.column_stack((est_start_time, est_pp_points))
#plt.plot(est_table[:,0], est_table[:,1])
#plt.xlim(time_comparison[first_user]["start"], time_comparison[first_user]["end"])
#plt.ylim(5000,8000)

In [ ]:
#Visually compare the real and est user pp within the overlap timeframe
plt.plot(real_table[:,0], real_table[:,1], label="real")
plt.plot(est_table[:,0], est_table[:,1], label="est")
plt.xlim(time_comparison[first_user]["start"], time_comparison[first_user]["end"])
plt.ylim(5500,7500)
plt.legend()
plt.title('Real vs ets usesr pp within the overlap timeframe');

In [ ]:
#Get fifty points of real pp
x = []
real_within = real_table[real_table[:,0] >= time_comparison[first_user]["start"]]
real_within = real_within[real_within[:,0] <= time_comparison[first_user]["end"]]
real_xp = real_within[:,0]
real_fp = real_within[:,1]

begin = real_xp[0] 
end = real_xp[-1] 
intervals = 50 
real_date_list = [] 
delta = (end - begin)/intervals 
for i in range(1, intervals + 1): 
    real_date_list.append((begin+i*delta).timestamp())

k  = 0
for i in real_xp:
    real_xp[k] = i.timestamp()
    k+=1

real_fifty_points = np.interp(real_date_list,list(real_xp),list(real_fp))
real_fifty_points

In [ ]:
#Get fifty points of est pp
x = []
est_within = est_table[est_table[:,0] >= time_comparison[first_user]["start"]]
est_within = est_within[est_within[:,0] <= time_comparison[first_user]["end"]]
xp = est_within[:,0]
fp = est_within[:,1]

begin = xp[0] 
end = xp[-1] 
intervals = 50 
date_list = [] 
delta = (end - begin)/intervals 
for i in range(1, intervals + 1): 
    date_list.append((begin+i*delta).timestamp())

k  = 0
for i in xp:
    xp[k] = i.timestamp()
    k+=1

est_fifty_points = np.interp(date_list,list(xp),list(fp))
est_fifty_points

In [ ]:
#Calculate mse for one user
mse = (np.square(real_fifty_points - est_fifty_points)).mean()
mse

In [ ]:
#Get 10k points of real pp
x = []
real_within = real_table[real_table[:,0] >= time_comparison[first_user]["start"]]
real_within = real_within[real_within[:,0] <= time_comparison[first_user]["end"]]
real_xp = real_within[:,0]
real_fp = real_within[:,1]

begin = real_xp[0] 
end = real_xp[-1] 
intervals = 10000 
real_date_list = [] 
delta = (end - begin)/intervals 
for i in range(1, intervals + 1): 
    real_date_list.append((begin+i*delta).timestamp())

k  = 0
for i in real_xp:
    real_xp[k] = i.timestamp()
    k+=1

real_tenThousand_points = np.interp(real_date_list,list(real_xp),list(real_fp))
#real_tenThousand_points

In [ ]:
#Get 10k points of est pp
x = []
est_within = est_table[est_table[:,0] >= time_comparison[first_user]["start"]]
est_within = est_within[est_within[:,0] <= time_comparison[first_user]["end"]]
xp = est_within[:,0]
fp = est_within[:,1]

begin = xp[0] 
end = xp[-1] 
intervals = 10000 
date_list = [] 
delta = (end - begin)/intervals 
for i in range(1, intervals + 1): 
    date_list.append((begin+i*delta).timestamp())

k  = 0
for i in xp:
    xp[k] = i.timestamp()
    k+=1

est_tenThousand_points = np.interp(date_list,list(xp),list(fp))
#est_tenThousand_points

In [ ]:
#Get area between curve for real and curve for est
a = time_comparison[first_user]["start"].timestamp()
b = time_comparison[first_user]["end"].timestamp()
N = 1000 #arbitrary number
dx = (b - a) / N
x_midpoint = np.linspace(dx / 2, b - dx / 2, N)
real_midpoint_riemann = np.sum(real_tenThousand_points * dx)
est_midpoint_riemann = np.sum(est_tenThousand_points * dx)
area_between_curves = real_midpoint_riemann - est_midpoint_riemann
area_between_curves

In [ ]:
#DTW
def dtw(s, t):
    n, m = len(s), len(t)
    dtw_matrix = np.zeros((n+1, m+1))
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix
cost_matrix = dtw(real_within[:,1], est_within[:,1])
dtw_distance = cost_matrix[-1][-1]